In [1]:
# IMPORTING LIBRARIES
from matplotlib import pyplot as plt
#from matplotlib.colors import ListedColormap
#from mlxtend.plotting import plot_decision_regions
#import seaborn as sb # visualizations
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [5]:
# LOADING DATASETS

datasets = ['datasets/Monday-WorkingHours.pcap_ISCX.csv','datasets/Tuesday-WorkingHours.pcap_ISCX.csv',
            'datasets/Wednesday-WorkingHours.pcap_ISCX.csv','datasets/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
            'datasets/Thursday-WorkingHours-Afternoon-Infiltration.pcap_ISCX.csv','datasets/Friday-WorkingHours-Morning.pcap_ISCX.csv',
            'datasets/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv','datasets/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv']

d1 = pd.read_csv(datasets[0])
d2 = pd.read_csv(datasets[1])
d3 = pd.read_csv(datasets[2])
d4 = pd.read_csv(datasets[3])
d5 = pd.read_csv(datasets[4])
d6 = pd.read_csv(datasets[5])
d7 = pd.read_csv(datasets[6])
d8 = pd.read_csv(datasets[7])

frames = [d1, d2, d3, d4, d5, d6, d7, d8]
main_df = pd.concat(frames)

In [11]:
# CLEANING DATA

col_name_consistency = {
'Flow ID': 'Flow ID',
'Source IP': 'Source IP',
'Src IP':  'Source IP',
'Source Port': 'Source Port',
'Src Port': 'Source Port',
'Destination IP': 'Destination IP',
'Dst IP': 'Destination IP',
'Destination Port': 'Destination Port',
'Dst Port': 'Destination Port',
'Protocol': 'Protocol',
'Timestamp': 'Timestamp',
'Flow Duration': 'Flow Duration',
'Total Fwd Packets': 'Total Fwd Packets',
'Tot Fwd Pkts': 'Total Fwd Packets',
'Total Backward Packets': 'Total Backward Packets',
'Tot Bwd Pkts': 'Total Backward Packets',
'Total Length of Fwd Packets': 'Fwd Packets Length Total',
'TotLen Fwd Pkts': 'Fwd Packets Length Total',
'Total Length of Bwd Packets': 'Bwd Packets Length Total',
'TotLen Bwd Pkts': 'Bwd Packets Length Total',
'Fwd Packet Length Max': 'Fwd Packet Length Max',
'Fwd Pkt Len Max': 'Fwd Packet Length Max',
'Fwd Packet Length Min': 'Fwd Packet Length Min',
'Fwd Pkt Len Min': 'Fwd Packet Length Min',
'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
'Fwd Pkt Len Mean': 'Fwd Packet Length Mean',
'Fwd Packet Length Std': 'Fwd Packet Length Std',
'Fwd Pkt Len Std': 'Fwd Packet Length Std',
'Bwd Packet Length Max': 'Bwd Packet Length Max',
'Bwd Pkt Len Max': 'Bwd Packet Length Max',
'Bwd Packet Length Min': 'Bwd Packet Length Min',
'Bwd Pkt Len Min': 'Bwd Packet Length Min',
'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
'Bwd Pkt Len Mean': 'Bwd Packet Length Mean',
'Bwd Packet Length Std': 'Bwd Packet Length Std',
'Bwd Pkt Len Std': 'Bwd Packet Length Std',
'Flow Bytes/s': 'Flow Bytes/s',
'Flow Byts/s': 'Flow Bytes/s',
'Flow Packets/s': 'Flow Packets/s',
'Flow Pkts/s': 'Flow Packets/s',
'Flow IAT Mean': 'Flow IAT Mean',
'Flow IAT Std': 'Flow IAT Std',
'Flow IAT Max': 'Flow IAT Max',
'Flow IAT Min': 'Flow IAT Min',
'Fwd IAT Total': 'Fwd IAT Total',
'Fwd IAT Tot': 'Fwd IAT Total',
'Fwd IAT Mean': 'Fwd IAT Mean',
'Fwd IAT Std': 'Fwd IAT Std',
'Fwd IAT Max': 'Fwd IAT Max',
'Fwd IAT Min': 'Fwd IAT Min',
'Bwd IAT Total': 'Bwd IAT Total',
'Bwd IAT Tot': 'Bwd IAT Total',
'Bwd IAT Mean': 'Bwd IAT Mean',
'Bwd IAT Std': 'Bwd IAT Std',
'Bwd IAT Max': 'Bwd IAT Max',
'Bwd IAT Min': 'Bwd IAT Min',
'Fwd PSH Flags': 'Fwd PSH Flags',
'Bwd PSH Flags': 'Bwd PSH Flags',
'Fwd URG Flags': 'Fwd URG Flags',
'Bwd URG Flags': 'Bwd URG Flags',
'Fwd Header Length': 'Fwd Header Length',
'Fwd Header Len': 'Fwd Header Length',
'Bwd Header Length': 'Bwd Header Length',
'Bwd Header Len': 'Bwd Header Length',
'Fwd Packets/s': 'Fwd Packets/s',
'Fwd Pkts/s': 'Fwd Packets/s',
'Bwd Packets/s': 'Bwd Packets/s',
'Bwd Pkts/s': 'Bwd Packets/s',
'Min Packet Length': 'Packet Length Min',
'Pkt Len Min': 'Packet Length Min',
'Max Packet Length': 'Packet Length Max',
'Pkt Len Max': 'Packet Length Max',
'Packet Length Mean': 'Packet Length Mean',
'Pkt Len Mean': 'Packet Length Mean',
'Packet Length Std': 'Packet Length Std',
'Pkt Len Std': 'Packet Length Std',
'Packet Length Variance': 'Packet Length Variance',
'Pkt Len Var': 'Packet Length Variance',
'FIN Flag Count': 'FIN Flag Count',
'FIN Flag Cnt': 'FIN Flag Count',
'SYN Flag Count': 'SYN Flag Count',
'SYN Flag Cnt': 'SYN Flag Count',
'RST Flag Count': 'RST Flag Count',
'RST Flag Cnt': 'RST Flag Count',
'PSH Flag Count': 'PSH Flag Count',
'PSH Flag Cnt': 'PSH Flag Count',
'ACK Flag Count': 'ACK Flag Count',
'ACK Flag Cnt': 'ACK Flag Count',
'URG Flag Count': 'URG Flag Count',
'URG Flag Cnt': 'URG Flag Count',
'CWE Flag Count': 'CWE Flag Count',
'CWE Flag Cnt': 'CWE Flag Count',
'ECE Flag Count': 'ECE Flag Count',
'ECE Flag Cnt': 'ECE Flag Count',
'Down/Up Ratio': 'Down/Up Ratio',
'Average Packet Size': 'Avg Packet Size',
'Pkt Size Avg': 'Avg Packet Size',
'Avg Fwd Segment Size': 'Avg Fwd Segment Size',
'Fwd Seg Size Avg': 'Avg Fwd Segment Size',
'Avg Bwd Segment Size': 'Avg Bwd Segment Size',
'Bwd Seg Size Avg': 'Avg Bwd Segment Size',
'Fwd Avg Bytes/Bulk': 'Fwd Avg Bytes/Bulk',
'Fwd Byts/b Avg': 'Fwd Avg Bytes/Bulk',
'Fwd Avg Packets/Bulk': 'Fwd Avg Packets/Bulk',
'Fwd Pkts/b Avg': 'Fwd Avg Packets/Bulk',
'Fwd Avg Bulk Rate': 'Fwd Avg Bulk Rate',
'Fwd Blk Rate Avg': 'Fwd Avg Bulk Rate',
'Bwd Avg Bytes/Bulk': 'Bwd Avg Bytes/Bulk',
'Bwd Byts/b Avg': 'Bwd Avg Bytes/Bulk',
'Bwd Avg Packets/Bulk': 'Bwd Avg Packets/Bulk',
'Bwd Pkts/b Avg': 'Bwd Avg Packets/Bulk',
'Bwd Avg Bulk Rate': 'Bwd Avg Bulk Rate',
'Bwd Blk Rate Avg': 'Bwd Avg Bulk Rate',
'Subflow Fwd Packets': 'Subflow Fwd Packets',
'Subflow Fwd Pkts': 'Subflow Fwd Packets',
'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
'Subflow Fwd Byts': 'Subflow Fwd Bytes',
'Subflow Bwd Packets': 'Subflow Bwd Packets',
'Subflow Bwd Pkts': 'Subflow Bwd Packets',
'Subflow Bwd Bytes': 'Subflow Bwd Bytes',
'Subflow Bwd Byts': 'Subflow Bwd Bytes',
'Init_Win_bytes_forward': 'Init Fwd Win Bytes',
'Init Fwd Win Byts': 'Init Fwd Win Bytes',
'Init_Win_bytes_backward': 'Init Bwd Win Bytes',
'Init Bwd Win Byts': 'Init Bwd Win Bytes',
'act_data_pkt_fwd': 'Fwd Act Data Packets',
'Fwd Act Data Pkts': 'Fwd Act Data Packets',
'min_seg_size_forward': 'Fwd Seg Size Min',
'Fwd Seg Size Min': 'Fwd Seg Size Min',
'Active Mean': 'Active Mean',
'Active Std': 'Active Std',
'Active Max': 'Active Max',
'Active Min': 'Active Min',
'Idle Mean': 'Idle Mean',
'Idle Std': 'Idle Std',
'Idle Max': 'Idle Max',
'Idle Min': 'Idle Min',
'Label': 'Label'
}
drop_columns = [ # this list includes all spellings across CIC NIDS datasets
    "Flow ID",    
    'Fwd Header Length.1',
    "Source IP", "Src IP",
    "Source Port", "Src Port",
    "Destination IP", "Dst IP",
    "Destination Port", "Dst Port",
    "Timestamp", "Bwd PSH Flags",
    "Bwd URG Flags", "Fwd Avg Bytes/Bulk",
    "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate",
    "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk",
    "Bwd Avg Bulk Rate"
]

main_df.columns = main_df.columns.str.strip()
main_df.drop(columns=drop_columns, inplace=True, errors='ignore')    
main_df.rename(columns=col_name_consistency, inplace=True)
main_df.replace([np.inf, -np.inf], np.nan, inplace=True)
main_df.dropna(inplace=True) 

In [12]:
# CREATING X & Y DATAFRAMES

X = main_df.drop(columns=['Label'])
y = main_df['Label'].replace(['BENIGN', 'Bot', 'PortScan', 'FTP-Patator', 'SSH-Patator', 'Web Attack � Brute Force', 
                              'Web Attack � Sql Injection', 'Web Attack � XSS', 'Heartbleed', 'DoS slowloris',
                              'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'DDoS' , 'Infiltration'], 
                             [0,1,1,2,2,2,3,3,3,4,4,4,4,4,4])

# 0 = None: Normal Traffic
# 1 = Low: Port Scans and Bots
# 2 = Medium: Bruteforce Attacks for FTP, SSH, and Web
# 3 = SQL, XSS, and Heartbleed Attacks
# 4 = DDos Attacks and Infiltration

In [13]:
# SPLITTING TRAIN / TEST

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# FITTING REGRESSION

knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn.fit(X_train, y_train)

pd.set_option('display.max_columns', None)
display(X_train)

/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated

,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Avg Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init Fwd Win Bytes,Init Bwd Win Bytes,Fwd Act Data Packets,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
260470,80244,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.000000,149.543891,24.923982,8.024400e+04,0.000000e+00,80244,80244,80244,8.024400e+04,0.000000e+00,80244,80244,0,0.000000e+00,0.000000e+00,0,0,0,0,40,0,24.923982,0.000000,6,6,6.000000,0.000000,0.000000e+00,0,0,0,0,1,0,0,0,0,9.000000,6.000000,0.000000,2,12,0,0,255,-1,1,20,0.0,0.000,0,0,0.0,0.0,0,0
5905,30211,1,1,45,110,45,45,45.000000,0.000000,110,110,110.000000,0.000000,5130.581576,66.201053,3.021100e+04,0.000000e+00,30211,30211,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,20,32,33.100526,33.100526,45,110,66.666667,37.527768,1.408333e+03,0,0,0,0,0,0,0,0,1,100.000000,45.000000,110.000000,1,45,1,110,-1,-1,0,20,0.0,0.000,0,0,0.0,0.0,0,0
246750,16,1,1,2,6,2,2,2.000000,0.000000,6,6,6.000000,0.000000,500000.000000,125000.000000,1.600000e+01,0.000000e+00,16,16,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,24,20,62500.000000,62500.000000,2,6,3.333333,2.309401,5.333333e+00,0,0,0,1,0,0,0,0,1,5.000000,2.000000,6.000000,1,2,1,6,1024,0,0,24,0.0,0.000,0,0,0.0,0.0,0,0
47092,95181057,183,235,11531,248734,1258,0,63.010929,181.646166,2658,0,1058.442553,588.240517,2734.420148,4.391630,2.282519e+05,3.226863e+06,58500000,0,95200000,5.229728e+05,4.876317e+06,58500000,0,95200000,4.066910e+05,4.304185e+06,58500000,1,0,0,3680,4712,1.922651,2.468979,0,2658,621.157518,672.969414,4.528878e+05,0,0,0,1,0,0,0,0,1,622.643541,63.010929,1058.442553,183,11531,235,248734,29200,54,182,20,3281140.5,4559198.958,6504981,57300,44300000.0,20100000.0,58500000,30100000
68707,94649856,2,1,8,0,8,0,4.000000,5.656854,0,0,0.000000,0.000000,0.084522,0.031696,4.730000e+07,6.690000e+07,94600000,78,94600000,9.460000e+07,0.000000e+00,94600000,94600000,0,0.000000e+00,0.000000e+00,0,0,1,0,64,32,0.021131,0.010565,0,8,4.000000,4.618802,2.133333e+01,0,1,0,0,1,0,0,0,0,5.333333,4.000000,0.000000,2,8,1,0,229,235,0,32,0.0,0.000,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26248,30818,1,1,66,98,66,66,66.000000,0.000000,98,98,98.000000,0.000000,5321.565319,64.897138,3.081800e+04,0.000000e+00,30818,30818,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,20,32,32.448569,32.448569,66,98,76.666667,18.475209,3.413333e+02,0,0,0,0,0,0,0,0,1,115.000000,66.000000,98.000000,1,66,1,98,-1,-1,0,20,0.0,0.000,0,0,0.0,0.0,0,0
40276,53882,4,4,176,400,44,44,44.000000,0.000000,100,100,100.000000,0.000000,10690.026350,148.472588,7.697429e+03,1.327532e+04,30894,3,53702,1.790067e+04,3.099880e+04,53695,3,22985,7.661667e+03,1.316734e+04,22866,48,0,0,80,80,74.236294,74.236294,44,100,68.888889,29.514591,8.711111e+02,0,0,0,0,0,0,0,0,1,77.500000,44.000000,100.000000,4,176,4

In [14]:
# PREDICT
y_pred = knn.predict(X_test)

/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [16]:
# CHECKING ACCURACY

print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

0.9735235748734132
0.049283148840767455
0.013536658246528903


/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/BelierJavier/Desktop/Vigil/env/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `

In [30]:
import pickle

pickle.dump(knn, open('ids_model','wb'))

In [4]:
#unit = np.array([1,80244,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,149.543891,24.923982,80244.0,0.0,80244,80244,
#                 80244.0,0.0,80244,80244,0,0.0,0.0,0,0,0,0,0,0,40,0,24.923982,0.0,6,6,6.0,0.0,0.0,0,0,
#                 0,0,1,0,0,0,0,9.0,6.0,0.0,0,0,0,0,0,0,2,12,0,0,255,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0])

#X_train.head(1)
#unitdf = pd.DataFrame(unit)
#predunit = unitdf.transpose()

#knn.predict(predunit)